In [12]:
import pickle
from utils.build_graph import build_graph_dgl,build_graph_pyg
import torch
def readFile(filePath):
    with open(f"./dataset/{dataset}/{filePath}.pkl", 'rb') as file:
        return pickle.loads(file.read())

In [13]:
dataset='quora'
Answers_train=readFile('Answers_train')
Answers_dev=readFile('Answers_dev')
Answers_test=readFile('Answers_test')
Questions=readFile('Questions')
Users=readFile('Users')

In [14]:
data,data_train_neg,data_dev_pos,data_dev_neg,data_test_pos,data_test_neg=build_graph_pyg(Users,Questions,Answers_train,Answers_dev,Answers_test)

In [15]:
data.to_homogeneous()

Data(edge_index=[2, 4762846], node_id=[556920], node_type=[556920], edge_type=[4762846])

In [16]:
homogeneous_data=data.to_homogeneous()
homogeneous_data.train_pos_edge_index=homogeneous_data.edge_index[:,homogeneous_data.edge_type==0]
homogeneous_data.edge_index=torch.concat([homogeneous_data.edge_index,data_dev_pos.to_homogeneous().edge_index,data_test_pos.to_homogeneous().edge_index],axis=1)
#homogeneous_data.edge_type=torch.concat([homogeneous_data.edge_type,data_dev_pos.to_homogeneous().edge_type,data_test_pos.to_homogeneous().edge_type])
homogeneous_data.val_pos_edge_index=data_dev_pos.to_homogeneous().edge_index
homogeneous_data.test_pos_edge_index=data_test_pos.to_homogeneous().edge_index
homogeneous_data.val_neg_edge_index=data_dev_neg.to_homogeneous().edge_index
homogeneous_data.test_neg_edge_index=data_test_neg.to_homogeneous().edge_index
homogeneous_data.train_neg_edge_index=data_train_neg.to_homogeneous().edge_index
homogeneous_data.x=torch.rand(homogeneous_data.node_id.shape[0],32)
homogeneous_data.y=homogeneous_data.node_type

In [17]:
def mask_to_index(index, size):
    all_idx = np.arange(size)
    return all_idx[index]

def index_to_mask(index, size):
    mask = torch.zeros((size, ), dtype=torch.bool)
    mask[index] = 1
    return mask
def random_coauthor_amazon_splits(data, num_classes, lcc_mask):
    """https://github.com/mengliu1998/DeeperGNN/blob/master/DeeperGNN/train_eval.py
    Set random coauthor/co-purchase splits:
    * 20 * num_classes labels for training
    * 30 * num_classes labels for validation
    rest labels for testing
    """
    indices = []
    if lcc_mask is not None:
        for i in range(num_classes):
            index = (data.y[lcc_mask] == i).nonzero().view(-1)
            index = index[torch.randperm(index.size(0))]
            indices.append(index)
    else:
        for i in range(num_classes):
            index = (data.y == i).nonzero().view(-1)
            index = index[torch.randperm(index.size(0))]
            indices.append(index)

    train_index = torch.cat([i[:20] for i in indices], dim=0)
    val_index = torch.cat([i[20:50] for i in indices], dim=0)

    rest_index = torch.cat([i[50:] for i in indices], dim=0)
    rest_index = rest_index[torch.randperm(rest_index.size(0))]

    data.train_mask = index_to_mask(train_index, size=data.num_nodes)
    data.val_mask = index_to_mask(val_index, size=data.num_nodes)
    data.test_mask = index_to_mask(rest_index, size=data.num_nodes)

In [18]:
from torch_geometric.utils import contains_isolated_nodes, remove_isolated_nodes
contains_isolated_nodes(homogeneous_data.edge_index)

True

In [19]:
if contains_isolated_nodes(homogeneous_data.edge_index):
    homogeneous_data.edge_index=remove_isolated_nodes(homogeneous_data.edge_index)[0]

In [20]:
homogeneous_data

Data(edge_index=[2, 4766846], node_id=[556920], node_type=[556920], edge_type=[4762846], train_pos_edge_index=[2, 202868], val_pos_edge_index=[2, 2000], test_pos_edge_index=[2, 2000], val_neg_edge_index=[2, 8000], test_neg_edge_index=[2, 8000], train_neg_edge_index=[2, 1622944], x=[556920, 32], y=[556920])

In [21]:
random_coauthor_amazon_splits(homogeneous_data,homogeneous_data.y.max()+1,None)

In [22]:
torch.save(homogeneous_data, f"./dataset/{dataset}/data.pt")